# Simpel Classification models
In deze notebook wordt simpele voorspellingsmodellen gemaakt en getest. Het gaat hier om een classificaitie probleem: Het doel is om te voorspellen of er in een recept iets met tomaat (dus tomaten, maar ook tomatenpuree en zongedroogde tomaat bijvoorbeeld) zit.

### Imports

In [1]:
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import random
from sklearn.metrics import recall_score, confusion_matrix, accuracy_score

### Data inlezen
We werken met de file ingredients, want daar staan alle recepten met ingredienten in. In dit model kijken we namelijk alleen naar de andere ingredienten uit een recept en niet naar tags of iets dergelijks.

In [2]:
ingredients_df = pd.read_csv(f"/data/foodboost/ingredients.csv", index_col = 0)
ingredients_df

,recipe,ingredient,quantity,unit
0,Kruidnoten met choco-discodip,melkchocolade,100.0,g
1,Kruidnoten met choco-discodip,kruidnoten,100.0,g
2,Kruidnoten met choco-discodip,discodip,2.0,el
3,Kruidnoten in marsepein,blanke marsepein,150.0,g
4,Kruidnoten in marsepein,ongezouten roomboter,15.0,g
...,...,...,...,...
71804,Biefstuk met rodewijnsaus en ham,olijfolie,4.0,el
71805,Biefstuk met rodewijnsaus en ham,biefstukken,4.0,NaN
71806,Biefstuk met rodewijnsaus en ham,boter,25.0,g
71807,Biefstuk met rodewijnsaus en ham,serranoham,4.0,plakken


### Onderzoeken naar tomaat in data
Eerst gaan we de beschikbare data bestuderen. Hoe vaak komen er recepten voor die iets met tomaat bevatten

In [3]:
ingredients_count = ingredients_df[ingredients_df['ingredient'] == 'tomaat']
len(ingredients_count)

145

In [4]:
list_tomaat = [a for a in ingredients_df['ingredient'] if "toma" in str(a)]
len(list_tomaat)

1880

In [5]:
u_list_tomaat = [*set(list_tomaat)]
len(u_list_tomaat)

194

Er komt dus 145 keer tomaat en 1880 keer een ingredient voor dat toma bevat, waar 194 verschillende ingredienten onder vallen. Dit zal niet altijd iets tomaatachtigs zijn, maar het grootste deel van de tijd wel. In dit voorbeeld worden al die ingredienten onder 1 categorie 'variant tomaat' geplaatst en gaat het model zometeen voorspellen of dit in een recept voorkomt.

In [6]:
# alle ingredienten met 'toma' vervangen voor 'variant tomaat'
for i in range(len(ingredients_df)):
    if ingredients_df.iloc[i][1] in u_list_tomaat:
        ingredients_df['ingredient'] = ingredients_df['ingredient'].replace(ingredients_df.iloc[i][1], 'variant tomaat')

In [7]:
ingredients_df[ingredients_df['ingredient'] == 'variant tomaat']

,recipe,ingredient,quantity,unit
125,Pico de gallo,variant tomaat,750.0,g
161,Koe loe kai,variant tomaat,700.0,g
165,Pasta pesto met kipstuckjes en tomatensalade,variant tomaat,5.0,NaN
188,Zoete aardappelstamppot met spruiten en vega g...,variant tomaat,195.0,g
194,'Kapsalon' met&nbsp;vegetarische&nbsp;kipshoar...,variant tomaat,250.0,g
...,...,...,...,...
71691,Ovenpasta met ei,variant tomaat,400.0,g
71699,Ratatouille met kabeljauwfilet,variant tomaat,250.0,g
71725,Romige spaghetti met zalmreepjes,variant tomaat,250.0,g
71750,Gegrilde auberginepakketjes van Esther,variant tomaat,2.0,NaN


### Omzetten van categorische data naar dummy data
Hieronder wordt een matrix gecreëerd met 0en en 1en. Hier staan de recepten als rijen en de kolommen zijn alle ingredienten die voorkomen in deze recepten. Deze data voorbereiding moet worden gedaan zodat het classificatie model met de data kan werken, want die moet nummerieke data hebben.

In [8]:
# functie matrix maken bij df
def make_matrix(df, list_indexnames, list_columnnames, index_col_indexnames, index_col_columnnames):
    matrix = pd.DataFrame(0, list_indexnames, list_columnnames)
    for i in range(len(df)):
        a = df.iloc[i][int(index_col_columnnames)]
        b = df.iloc[i][int(index_col_indexnames)]
        index_a = list_columnnames.index(a)
        index_b = list_indexnames.index(b)
        matrix.iloc[index_b][index_a] = 1
    return matrix

In [9]:
ingredients_list = ingredients_df['ingredient'].unique().tolist()
recipes_list = ingredients_df['recipe'].unique().tolist()

ingr_matrix = make_matrix(ingredients_df, recipes_list, ingredients_list, 0, 1)

In [10]:
ingr_matrix

,melkchocolade,kruidnoten,discodip,blanke marsepein,ongezouten roomboter,nougatine in bakje,pure chocolade,cacaopoeder,hagelslag puur,witte basterdsuiker,...,goudreinetten,gemengde kruiden,Garden Gourmet falafelburger,prei a la creme deelblokjes,boomgaardsap peer,zalmreepjes,rodekool met appel,friszoete appels,winterpenen,Blooker cacaopoeder
Kruidnoten met choco-discodip,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kruidnoten in marsepein,0,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kruidnoten met chocodips,0,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Pepernotentaart met marsepeinstrik,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Perencake,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Paddenstoelen en courgettegratin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Peren-amandelcoupe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ceviche met sint-jakobsschelpen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pittige truffels,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


### Model en Trainen
We creëren een doelvariabele: y en voorspellende variabele: X. Ook wordt de data in een train en test set gesplitst. Daarna gaan we het model fitten op de traindata.

de y is een kolom met de info of er wel of niet 'variant tomaat' in het recept zit:

In [11]:
# 0 als variant tomaat er niet in zit, 1 als het er wel in zit
y = ingr_matrix['variant tomaat'].to_frame()
y

,variant tomaat
Kruidnoten met choco-discodip,0
Kruidnoten in marsepein,0
Kruidnoten met chocodips,0
Pepernotentaart met marsepeinstrik,0
Perencake,0
...,...
Paddenstoelen en courgettegratin,0
Peren-amandelcoupe,0
Ceviche met sint-jakobsschelpen,0
Pittige truffels,0


X zijn alle andere ingredienten van de recepten, dus de df/matrix zonder de y-kolom.

In [12]:
# dataframe met 1en en 0en
X = ingr_matrix.drop('variant tomaat', axis = 1)
X

,melkchocolade,kruidnoten,discodip,blanke marsepein,ongezouten roomboter,nougatine in bakje,pure chocolade,cacaopoeder,hagelslag puur,witte basterdsuiker,...,goudreinetten,gemengde kruiden,Garden Gourmet falafelburger,prei a la creme deelblokjes,boomgaardsap peer,zalmreepjes,rodekool met appel,friszoete appels,winterpenen,Blooker cacaopoeder
Kruidnoten met choco-discodip,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kruidnoten in marsepein,0,1,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Kruidnoten met chocodips,0,1,0,0,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
Pepernotentaart met marsepeinstrik,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
Perencake,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Paddenstoelen en courgettegratin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Peren-amandelcoupe,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Ceviche met sint-jakobsschelpen,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Pittige truffels,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


We creëeren een train (hier 70% van de data) en test (30%) set van X en y. 

In [13]:
# random splitten in train en test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

1. Logistic Regression model

In [14]:
log_reg = LogisticRegression()

In [15]:
log_reg.fit(X_train, y_train)

/opt/jupyterhub/anaconda/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [16]:
# hieronder laten we het model voorspellingen voor y doen op de testset. y_pred is dus een lijst met 0en en 1en
y_pred_lg = log_reg.predict(X_test)
y_pred_lg

array([0, 0, 0, ..., 0, 0, 1])

2. K-Nearest Neighbors

In [17]:
KNN = KNeighborsClassifier()

In [18]:
KNN.fit(X_train, y_train)

/opt/jupyterhub/anaconda/lib/python3.9/site-packages/sklearn/neighbors/_classification.py:207: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier()

In [19]:
# hieronder laten we het model voorspellingen voor y doen op de testset. y_pred is dus een lijst met 0en en 1en
y_pred_KNN = KNN.predict(X_test)
y_pred_KNN

array([0, 0, 0, ..., 0, 0, 0])

3. Desicion Tree

In [20]:
D_Tree = DecisionTreeClassifier()

In [21]:
D_Tree.fit(X_train, y_train)

DecisionTreeClassifier()

In [22]:
# hieronder laten we het model voorspellingen voor y doen op de testset. y_pred is dus een lijst met 0en en 1en
y_pred_tree = D_Tree.predict(X_test)
y_pred_tree

array([0, 0, 0, ..., 0, 0, 1])

## Testen
Deze modellen gaan we testen op de testdata, door te kijken naar accuracy (percentage goed verspelde waardes) en recall score (deel goed voorspelde 1en/positives van alle 1en/postives).

1. Logistic Regression: "y_pred_lg"

In [23]:
# accuracy
accuracy_score(y_test, y_pred_lg)

0.8085758039816233

In [24]:
# conf matrix: TN, FP
#              FN, TP
confusion_matrix(y_test, y_pred_lg)

array([[1955,  124],
       [ 376,  157]])

In [25]:
# recall = TP/(TP + FN)
recall_score(y_test, y_pred_lg)
# Na te rekenen uit confusion matrix: 157/(376+157)

0.2945590994371482

2. K-Nearest Neighbour: "y_pred_KNN"

In [26]:
# accuracy
accuracy_score(y_test, y_pred_KNN)

0.7947932618683001

In [27]:
# conf matrix:
confusion_matrix(y_test, y_pred_KNN)

array([[1995,   84],
       [ 452,   81]])

In [28]:
# recall = TP/(TP + FN)
recall_score(y_test, y_pred_KNN)

0.15196998123827393

3. Desicion Tree: "y_pred_tree"

In [29]:
# accuracy
accuracy_score(y_test, y_pred_tree)

0.774885145482389

In [30]:
# conf matrix:
confusion_matrix(y_test, y_pred_tree)

array([[1886,  193],
       [ 395,  138]])

In [31]:
# recall = TP/(TP + FN)
recall_score(y_test, y_pred_tree)

0.2589118198874296

# Conclusie
Logistic Regression lijkt van deze 3 modellen het beste resultaat te geven. De accuracy is daar 0.81, dat betekent dat het model 81% van de tijd het goede voorspelt voor een recept. De recall score is echter maar 0.29. Dit betekent dat het model vaak voorspeld dat er geen tomaat o.i.d. in zit, terwijl dit wel het geval is. In maar 1880 van de 8706 recepten komt iets met tomaat voor, daarom is de accuracy nog redelijk hoog: Het model heeft het vaak wel goed als die 0 voorspeld. Dit model is dus niet heel goed.

(Het samenvoegen van de ingrediënten met tomaat heeft niet veel toegevoegd en was niet nodig geweest)